In [288]:
%matplotlib inline
from ipywidgets import widgets
from ipywidgets import Layout
from IPython import display
import numpy as np
import serial
import time
import sys
import glob

N=8
N_out=8

data_input_style = "<style>.data_input input { background-color:#D0F0D0 !important; }"+\
                   ".data_wrong input { background-color:#F00000 !important; }</style>"

clk_s=100000000 #clock cycle per second

connected = False

p_d=["0" for n in range(10)]
p_w=["10e-6" for n in range(10)] 

def serial_ports():
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result



def send_pulser(cmd='', show=False):
    global ser
    ser.write(cmd)
    #print cmd
    out = b''
    while ser.inWaiting() > 0:
        out += ser.read(1)
    if out != '':
        if show:
            print (">>" + out)
    return out

def send_trigger(b):
    send_pulser(b"TRIG\r\n")
    
def send_delay(delay,pulser):
    send_pulser('SET DELAY {:d} {:d}\r\n'.format(int(delay*clk_s),pulser).encode())

def send_width(width,pulser):
    send_pulser('SET WIDTH {:d} {:d}\r\n'.format(int(width*clk_s),pulser).encode())     
    
def disable_ui(b):
    print("")
    
def my_connect(newvalue):
    global ser
    if (w_connect.value=='connect'):
        ser = serial.Serial(w_ports.value, baudrate=115200)
        time.sleep(1)
        connected = True
        disable_ui(False)
    else:
        ser.close()
        connected = False
        disable_ui(True)

def dw_change(change):
    delay=True
    try:
        i=w_d.index(change['owner'])
        inp_i=w_d[i]
        out_i=w_di[i]
        check = lambda a : (a<100) and (a>=0)      
    except ValueError:
        i=w_w.index(change['owner'])
        inp_i=w_w[i]
        out_i=w_wi[i]
        check = lambda a : (a<100) and (a>=10e-9)
        delay=False
    try:
        myf=float(eval(change['new']))
        if check(myf):
            mys='{:{width}.{prec}f}'.format(myf, width=12, prec=9)
            if delay:
                send_delay(myf,i)
            else:
                send_width(myf,i)
        else:
            raise
    except:
        inp_i.remove_class('data_input')
        inp_i.add_class('data_wrong')
        pass
    else:
        out_i.value=mys
        inp_i.remove_class('data_wrong')
        inp_i.add_class('data_input')

def en_change(change):
    #i=w_e.index(change['owner'])
    b=0;
    for i in range(N):
        b+=(w_e[i].value==True)*2**i
    send_pulser('SET ENABLE {:d}\r\n'.format(b).encode())
    print(b)
        
try:
    myserialports
except NameError:
    myserialports=serial_ports()

w_connect=widgets.ToggleButtons(description='Pulser:',\
                               options=['connect', 'disconnect'], value="disconnect")
w_ports=widgets.Dropdown(
    options=myserialports,
    value=myserialports[-1],
    description='Port:',
    disabled=False,
)

w_connect.observe(my_connect, names='value')

w_l=[]; w_e=[]; w_d=[]; w_di=[]; w_w=[]; w_wi=[]
for n in range(N):
    #Label
    w_l.append(widgets.Label('P{:d}'.format(n)))
    w_l[n].layout.width="3ex"
    #Enable
    w_e.append(widgets.Checkbox(value=False, disabled=False, indent=False))
    w_e[n].layout.max_width="3ex"
    w_e[n].observe(en_change, names='value')
    #Input Delay
    w_d.append(widgets.Text(value=p_d[n],placeholder='Pulser 1',disabled=False))
    w_d[n].layout.max_width="20ex"
    w_d[n].observe(dw_change, names='value')
    w_d[n].add_class('data_input')
    #Evaled Delay
    w_di.append(widgets.Text(value=p_d[n],placeholder='Type something',disabled=False))
    w_di[n].layout.max_width="20ex"

    #Input Width
    w_w.append(widgets.Text(value=p_d[n],placeholder='Pulser 1',disabled=False))
    w_w[n].layout.max_width="20ex"
    w_w[n].observe(dw_change, names='value')
    w_w[n].add_class('data_input')
    #Evaled Width
    w_wi.append(widgets.Text(value=p_w[n],placeholder='Type something',disabled=False))
    w_wi[n].layout.max_width="20ex"

disable_ui(True)
w_pps=[]
for n in range(N):
    w_pps.append(widgets.VBox([widgets.HBox([w_l[n],w_d[n],w_w[n]]),\
                              widgets.HBox([w_e[n],w_di[n],w_wi[n]])]));
    
w_trigger=widgets.Button(description='Trigger', button_style='danger')
w_trigger.on_click(send_trigger)

ht=widgets.HTML(data_input_style)
display.display(widgets.HBox([ht,w_ports,w_connect,w_trigger]))
for n in range(int(N/2)):
    display.display(widgets.HBox([w_pps[n*2+i] for i in range(2)]))


128
0
128
0
128
129
133


In [257]:
w_ports.value

'/dev/tty.usbmodem1421'

In [43]:
ser.write(b"TRIG\r\n")

6

In [23]:
ser.write(b"SET WIDTH 300000000 7\r\n")

23

In [32]:
widgets.HBox??

In [56]:
ser.write(b"REQ PULSE\n\r")

11

In [116]:
w_d0.value='1233'

In [62]:
print("{:,}".format(1000000000))

1,000,000,000


In [63]:
"{:,}".format(1000000000)

'1,000,000,000'

In [97]:
"{:,}".format(p_w[0])

'10,000'

In [90]:
type(np.int32(11))

numpy.int32

In [107]:
p_d

['1', '1', '1', '1', '1', '1', '1', '1', '1', '1']

In [122]:
a={'name': 'value', 'old': '1', 'new': '13'}

In [133]:
w_d0.remove_class


In [171]:
w_d=[]

w_d.append(widgets.Text(value=p_d[0],placeholder='Pulser 1',disabled=False))

In [284]:
w_e[0].value

False

In [209]:
l=[x for x in w_d if x == a]

In [210]:
l

[Text(value='', layout=Layout(max_width='20ex'), placeholder='Pulser 1', _dom_classes=('data_input',))]

In [211]:
w_d.index(a)

0

In [212]:
a

Text(value='', layout=Layout(max_width='20ex'), placeholder='Pulser 1', _dom_classes=('data_input',))

In [276]:
myf=1;i=2
print('SET DELAY {:d} {:d}\r\n'.format(int(myf*1e9),i))

SET DELAY 1000000000 2



In [279]:
myf=1;i=2
send_pulser('SET DELAY {:d} {:d}\r\n'.format(int(myf*1e9),i))

AttributeError: 'bytes' object has no attribute 'format'

In [280]:
'SET DELAY {:d} {:d}\r\n'.format(int(myf*1e9),i).encode()

b'SET DELAY 1000000000 2\r\n'